# Run all the experiments for the MRMR paper

Performs all the experiments. 

In [1]:
import sys
sys.path.append("../..")
from exp_run_config import Config
Config.PROJECTNAME = "WaterBerryFarms"

import pathlib
from pprint import pprint
import gzip as compress
import pickle
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

import wbf_figures
from wbf_simulate import run_nrobot1day

from information_model import StoredObservationIM

In [2]:
experiment = "mrmr"
lookup = {"mrmr-clustered-1pio-2con": "mrmr", "mrmr-clustered-3lawnmowers": "lawn", "mrmr-clustered-3random": "rwp", "mrmr-unclustered-1pio-2con" : "mrmr",
"mrmr-unclustered-3lawnmowers": "lawn", "mrmr-unclustered-3random": "rwp"}

In [3]:
# the experiment for this notebook
expall = Config().get_experiment(experiment, "mrmr_all")

for run in expall["allruns"]:
    exp = Config().get_experiment(experiment, run)
    pprint(exp)
    run_nrobot1day(exp)



***ExpRun**: Loading pointer config file:
	C:\Users\lboloni\.config\WaterBerryFarms\mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	G:\My Drive\LotziStudy\Code\PackageTracking\WaterBerryFarms\settings\WBF-config-LotziYoga.yaml
***ExpRun**: No system dependent experiment file
	 G:\My Drive\LotziStudy\Code\PackageTracking\WaterBerryFarms\settings\experiment-config\LotziYoga\mrmr\mrmr_all_sysdep.yaml,
	 that is ok, proceeding.
***ExpRun**: Configuration for exp/run: mrmr/mrmr_all successfully loaded
***ExpRun**: No system dependent experiment file
	 G:\My Drive\LotziStudy\Code\PackageTracking\WaterBerryFarms\settings\experiment-config\LotziYoga\mrmr\mrmr-clustered-1pio-2con_sysdep.yaml,
	 that is ok, proceeding.
***ExpRun**: Configuration for exp/run: mrmr/mrmr-clustered-1pio-2con successfully loaded
Experiment: {'velocity': 1, 'timesteps-per-day': 1000, 'time-start-environment': 25, 'im_resolution': 10, 'experiment_name': 'mrmr', 'name': '1 pioneer 2 contractors clu

INFO:root:create_type_map shape=(10201,)
INFO:root:Infection matrix: [0.         0.25       0.4375     0.578125   0.68359375 0.76269531
 0.82202148 0.86651611 0.89988708 0.92491531 0.94368649 0.95776486
 0.96832365 0.97624274 0.98218205 0.98663654 0.9899774  0.99248305
 0.99436229 0.99577172 0.99682879 0.99762159 0.99821619 0.99866214
 0.99899661]
INFO:root:Infection matrix: [0.         0.15       0.2775     0.385875   0.47799375 0.55629469
 0.62285048 0.67942291 0.72750947 0.76838305 0.8031256  0.83265676
 0.85775824 0.87909451 0.89723033 0.91264578 0.92574891 0.93688658
 0.94635359 0.95440055 0.96124047 0.9670544  0.97199624 0.9761968
 0.97976728]
INFO:root:Environment.proceed - calling the inner_proceed
INFO:root:Environment.proceed - calling the inner_proceed
INFO:root:PrecalculatedEnvironment at timestamp 1
INFO:root:PrecalculatedEnvironment: file C:\Users\lboloni\Documents\Code\_TempData\WBF\ExperimentData\environment\mrmr-custom-clustered\precalc_tylcv\env_value_00001.gz already

policy-name
policy-code
seed
budget
[{'exp-policy': Experiment: {'experiment_name': 'policy', 'policy-name': 'mrmr-pioneer', 'policy-code': 'MRMR_Pioneer', 'run_name': 'generic', 'exp_run_sys_indep_file': 'C:\\Users\\lboloni\\Documents\\Code\\_Checkouts\\WBF\\WaterberryFarms\\experiment_configs\\policy\\generic.yaml', 'data_dir': 'C:\\Users\\lboloni\\Documents\\Code\\_TempData\\WBF\\ExperimentData\\policy\\generic', 'subrun_name': None, 'time_started': '2025-05-08 14:31:45', 'seed': 1, 'budget': 1000},
  'name': 'pio'},
 {'exp-policy': Experiment: {'experiment_name': 'policy', 'policy-name': 'mrmr-contractor-1', 'policy-code': 'MRMR_Contractor', 'run_name': 'generic', 'exp_run_sys_indep_file': 'C:\\Users\\lboloni\\Documents\\Code\\_Checkouts\\WBF\\WaterberryFarms\\experiment_configs\\policy\\generic.yaml', 'data_dir': 'C:\\Users\\lboloni\\Documents\\Code\\_TempData\\WBF\\ExperimentData\\policy\\generic', 'subrun_name': None, 'time_started': '2025-05-08 14:31:45', 'seed': 2, 'budget': 1

ValueError: list.remove(x): x not in list

In [ ]:
# load back all the results 
all_results = {}

for run in expall["allruns"]:
    exp = Config().get_experiment(experiment, run)
    pprint(exp)

    resultsfile = pathlib.Path(exp["data_dir"], "results.pickle")
    if not resultsfile.exists():
        print(f"Results file does not exist:\n{resultsfile}")
        print("Run the notebook Run-1Robot1Day with the same exp/run to create it.")
        raise Exception("Nothing to do.")

    # load the results file
    with compress.open(resultsfile, "rb") as f:
        results = pickle.load(f)    
    all_results[run] = results


In [ ]:
all_results.keys()

### Visualize the replanning process
* for all the mrmr scenarios, for the contractor agents
* for all the replanning points (which are the indexes of the oldplans dictionary)
* create a figure
* visualize the observations to that time with a continuous line
* visualize the rest of the plan from the oldplans

__FIXME__: There is some kind of problem about how the position of the robot is drawn. Somehow the robot position does not align with the end of the trajectory, no matter how I am taking the robot position from. May 8, 2025, I have turned this off, because it is more important to submit the paper, I will debug it later.

In [ ]:
def show_robot_with_plan(expall, scenario, results, robotno, t):
    """Visualize the plan of the robot at a certain time point"""

    ROBOT_COLORS = ["#E69F00", "#56B4E9", "#009E73"]
    robot_color = ROBOT_COLORS[2]

    robot = results["robots"][robotno]
    observations = [o[robotno] for o in results["observations"]]
    observations = observations[0:int(t)]
    if observations:
        print(f"Last observations: {observations[-1]}")

    oldplan = robot.oldplans[t]


    filename = f"plans_{scenario}_{robot.name}_{t}"

    fig, ax = plt.subplots(1,1, figsize=(3, 3))
    wbf_figures.show_env_tylcv(results, ax)
    # obs = observations[1:int(t)]
    # color = "blue"
    wbf_figures.show_individual_robot_path(results, ax, robot=robot, observations=observations, pathcolor=robot_color, pathwidth=1,  draw_robot=False, robotcolor=robot_color, from_obs=0, to_obs=int(t))

    # add the plan
    # print(f"Oldplan beginning: {oldplan[0]}")
    planx = [a["x"] for a in oldplan]
    plany = [a["y"] for a in oldplan]
    ax.add_line(matplotlib.lines.Line2D(planx, plany, color = robot_color, linestyle=":", linewidth=1))

    # position of the robot
    # ax.add_patch(matplotlib.patches.Circle((observations[int(t)]["x"], observations[int(t)]["y"]), radius=3, facecolor=robot_color))
    # ax.add_patch(matplotlib.patches.Circle((observations[-1]["x"], observations[-1]["y"]), radius=3, facecolor=robot_color))
    # ax.add_patch(matplotlib.patches.Circle((oldplan[0]["x"], oldplan[0]["y"]), radius=3, facecolor="yellow"))

    ax.set_title(f"{robot.name} at t={int(t)}")
    filepath = pathlib.Path(expall.data_dir(), f"{filename}.pdf")
    plt.savefig(filepath)
    print(f"Done saving to {filepath}")

In [ ]:
import logging
logging.getLogger("fontTools").setLevel(logging.WARNING)


scenario = "mrmr-clustered-1pio-2con"
results = all_results[scenario]
robotno = 2
oldplans = results["robots"][robotno].oldplans
#for t in oldplans:
#    print(t)
show_robot_with_plan(expall, scenario, results, robotno, 646.0)

In [ ]:
obs = results["observations"]
obs[4]

In [ ]:
for scenario in all_results:
    if "1pio" not in scenario: continue
    results = all_results[scenario]
    for i, robot in enumerate(results["robots"]):
        if "con" in robot.name:
            print(robot.name)
            # FIXME: I think that the problem here is that I won't have access to the policy here, so I will need to store the old plans somewhere else... maybe in the robot...?
            oldplans = robot.oldplans
            for t in oldplans:
                print(t)
                show_robot_with_plan(expall, scenario, results, i, t)

### Visualize detection paths 
* for all running scenarios
* create a graph for the visualization of the paths, with the visualize the detections
* create a bargraph with the number of detection points for each of them
* FIXME: maybe also visualize the money for the mrmr agents

In [ ]:
def show_robot_trajectories_and_detections(expall, name, results, robot_colors):

    fig, ax = plt.subplots(1,1, figsize=(3, 3))
    wbf_figures.show_env_tylcv(results, ax)
    ax.set_title(lookup[name])
    custom_lines = []
    labels = []

    for i, robot in enumerate(results["robots"]):
        color = robot_colors[i % len(results["robots"])]
        observations = [o[i] for o in results["observations"]]
        wbf_figures.show_individual_robot_path(results, ax, robot=robot, observations=observations, pathcolor=color, draw_robot=False)
        wbf_figures.show_individual_robot_detections(results, ax, robotno=i, detection_color=color, radius=0.5)
        # adding to the legend
        custom_lines.append(matplotlib.lines.Line2D([0], [0], color=color, lw=2))
        labels.append(robot.name)


    # Add both automatic and manual entries to the legend
    ax.legend(handles=[*custom_lines],
            labels=labels)    

    plt.savefig(pathlib.Path(expall.data_dir(), f"detections-map-{name}.pdf"))

In [ ]:
def count_detections(results, robotno, field = "TYLCV"):
    """Returns the number of detections for the specified robot, adapted from wbf_figures.show_detections"""
    obs = np.array(results["observations"])[:, robotno]
    detections = [[a[StoredObservationIM.X], a[StoredObservationIM.Y]] for a in obs if a[field][StoredObservationIM.VALUE] == 0.0]
    return len(detections)

In [ ]:
def show_agentwise_detections(expall, name, results, robot_colors):    
    fig, ax = plt.subplots(1,1, figsize=(3, 3))
    ax.set_title(lookup[name])
    total = 0
    for i, robot in enumerate(results["robots"]):
        detections = count_detections(results, i)
        total += detections
        br = ax.bar(robot.name, detections, color=robot_colors[i])
    ax.bar("Total", total, color="gray")
    plt.savefig(pathlib.Path(expall.data_dir(), f"detections-bar-{name}.pdf"))

In [ ]:
#ROBOT_COLORS = ["red", "blue", "green"]
ROBOT_COLORS = ["#E69F00", "#56B4E9", "#009E73"]
for scenario in all_results:
    results = all_results[scenario]
    name = scenario
    show_robot_trajectories_and_detections(expall, name, results, robot_colors=ROBOT_COLORS)    
    show_agentwise_detections(expall, name, results, robot_colors = ROBOT_COLORS)

### Overall pickings comparison graphs 
* compare the unclustered and clustered cases 
* create comparison graphs

In [ ]:
name_colors = ["#CC6666", "#9999CC", "#66CC99"]

In [ ]:
def show_comparative_detections(expall, name, values):    
    fig, ax = plt.subplots(1,1, figsize=(3, 3))
    for i, policyname in enumerate(values):
        br = ax.bar(lookup[policyname], values[policyname], color=name_colors[i])
    plt.savefig(pathlib.Path(expall.data_dir(), f"comparative-bar-{name}.pdf"))

In [ ]:
for group in ["clustered-runs", "unclustered-runs"]:
    values = {}
    for scenario in expall[group]:
        results = all_results[scenario]
        total = 0
        for i, robot in enumerate(results["robots"]):
            detections = count_detections(results, i)
            total += detections
        values[scenario] = total
    show_comparative_detections(expall, f"comparative-{group}", values)